In [ ]:
# Just the normal prequisites for using matplotlib and numpy in a Jupyter notebook
%matplotlib inline
# Use the svg backend, in my opinion it just makes better looking plots
%config InlineBackend.figure_format = 'svg'

import PySpice.Logging.Logging as Logging
logger = Logging.setup_logging( logging_level='CRITICAL')

import os
import sys
from pathlib import Path
import IPython
from scipy import signal
import scipy.fftpack
from PySpice.Unit import *
from PySpice.Spice.Parser import SpiceParser
from PySpice.Spice.Netlist import Circuit, SubCircuit, SubCircuitFactory
from PySpice.Spice.Library import SpiceLibrary
from PySpice.Probe.Plot import plot
from PySpice.Doc.ExampleTools import find_libraries
from PySpice.Math import *
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import unittest

import PySpice.Logging.Logging as Logging
logger = Logging.setup_logging()

directory_path = Path(os.path.abspath('')).resolve().parent.parent
spice_libraries_path = directory_path.joinpath("lib", "spice")
spice_library = SpiceLibrary(spice_libraries_path)

directory_path = Path(os.path.abspath('')).resolve()

![module](https://img.shields.io/badge/module-vca-yellow) ![status](https://img.shields.io/badge/status-work%20in%20progress-orange)

## *pegel*

<a href="https://photos.app.goo.gl/pg6iZUC32rTKX5LC7"><img src="https://spielhuus.github.io/elektrophon/images/kontrast-logo-tmb.jpg" height="300px" align="right"></a>

***table of contents***

* [*about*](#about)
* [*construction*](#construction)
* [*calibration*](#calibration)
* [*usage*](#usage)
* [*credits*](#credits)
* [*links*](#links)
* [*changelog*](#changelog)
  
<br/><br/><br/><br/>

## *about*

pegel is a voltage controlled amplifier (vca). there are various designs to implement a vca. rod elliott (ESP) has an article on vca techniques [[1][1]]. diy synthesizer modules are usualy designed with an integrated vca chip [[2][2]]  or with a differential amplifier [[3][3]]. the integrated vca chips are either obsolete or rather expensive. the differential amplifier can be built with transistors only. the design has some downsides. even in the simulation the result is not symetryc and has a dc offset from the control voltage. this needs a lot of trimming to get an accurate result. but the biggest downside is, that this design can not do proper amplitude modulation (am). when the carrier signal goes below zero the base signal is completely muted. 

In [ ]:
#load the diffpair schema
kicad_netlist_path = directory_path.joinpath('diffpair', 'diffpair.cir')
parser = SpiceParser(path=str(kicad_netlist_path))

In [ ]:
#simulate diffpair with envelope
diffpair_envelope = parser.build_circuit(ground=5)
diffpair_envelope.include(spice_library['BC547B'])
diffpair_envelope.V('1', '+15V', diffpair_envelope.gnd, 'DC 15')
diffpair_envelope.V('2', '-15V', diffpair_envelope.gnd, 'DC -15')
diffpair_envelope.V('3', 'Vin_a', diffpair_envelope.gnd, 'DC 0V AC 0V SIN(0 25m 1k)')
diffpair_envelope.V('4', 'Vin_b', diffpair_envelope.gnd, 'DC 0V AC 0V PULSE(200m -4.5 1m 1u 15m 1u)')
simulator = diffpair_envelope.simulator(temperature=25, nominal_temperature=25)
analysis_envelope = simulator.transient(step_time=1@u_us, end_time=20@u_ms)

In [ ]:
#simulate  amplitude modulation
diffpair_am = parser.build_circuit(ground=5)
diffpair_am.include(spice_library['BC547B'])
diffpair_am.V('1', '+15V', diffpair_am.gnd, 'DC 15')
diffpair_am.V('2', '-15V', diffpair_am.gnd, 'DC -15')
diffpair_am.V('3', 'Vin_a', diffpair_am.gnd, 'DC 0V AC 0V SIN(0 25m 1k)')
diffpair_am.V('4', 'Vin_b', diffpair_am.gnd, 'DC 0V AC 0V SIN(0 5 100)')

simulator = diffpair_am.simulator(temperature=25, nominal_temperature=25)
analysis_am = simulator.transient(step_time=1@u_us, end_time=20@u_ms)

In [ ]:
#plot the results
fig, (ax0, ax1, ax2) = plt.subplots(nrows=1, ncols=3, sharex=False, figsize=(18, 6))

#plot the image
img_diffpair_path = directory_path.joinpath('diffpair', 'diffpair.png')
img_diffpair = mpimg.imread(str(img_diffpair_path))
im = ax0.imshow(img_diffpair)
ax0.axis('off')
ax0.set_title('long tailed pair differential amplifier', y=-0.2)
        
#plot with envelope
ax1.plot(analysis_envelope['Vin_b'].abscissa*1000, analysis_envelope['Vin_b'] * 0.2 * -1, c='grey')  # envelope input (scaled)
ax1.plot(analysis_envelope['Vout_b'].abscissa*1000, analysis_envelope['Vout_b'] - analysis_envelope['Vout_a'], c='orange')  # differential output
ax1.grid()
ax1.set_xlabel('t [ms]')
ax1.set_ylabel('[V]')
ax1.set_title('output with a cv envelope', y=-0.2)

#plot the amplitude modulation
ax2.plot(analysis_am['Vout_b'].abscissa*1000, analysis_am['Vout_b'] - analysis_am['Vout_a'], c='orange')  # differential output
ax2.plot(analysis_am['Vout_b'].abscissa*1000, analysis_am['Vin_b'] * 0.2 * -1, c='grey')  # differential output
ax2.grid()
ax2.set_xlabel('t [ms]')
ax2.set_ylabel('[V]')
ax2.set_title('output with a sine control voltage.', y=-0.2)
plt.show()


more promising is it to use a four quadrant multiplier, also known as gilbert cell. the gilbert cell is designed around two differential amplifiers. the carrier signal switches between those. the result is a multiplication of the input voltages. the gilbert cell is mostly used for amplitude modulation in radio transmission. if you look at the integratd circuits like the 633 it can modulate signals up in the gigahertz range. the gilbert cell is not described in all details here, there is a good introduction from w2aew [[4][4]].




In [ ]:
#load the gilbert_cell schema
kicad_netlist_path = directory_path.joinpath('gilbert_cell_2', 'gilbert_cell.cir')
parser = SpiceParser(path=str(kicad_netlist_path))

In [ ]:
#simulate with envelope
gilbert_cell_envelope = parser.build_circuit(ground=5)
gilbert_cell_envelope.include(spice_library['BC547B'])
gilbert_cell_envelope.include(spice_library['TL072'])

gilbert_cell_envelope.V('1', '+15V', gilbert_cell_envelope.gnd, 'DC 15')
gilbert_cell_envelope.V('2', '-15V', gilbert_cell_envelope.gnd, 'DC -15')
gilbert_cell_envelope.V('3', 'AUDIO_IN', gilbert_cell_envelope.gnd, 'DC 0V AC 0V SIN(0 10m 1k)')
gilbert_cell_envelope.V('4', 'CV_IN', gilbert_cell_envelope.gnd, 'DC 0V AC 0V PULSE(0 20m 1m 0u 10m 1u)')
gilbert_cell_envelope.V('5', '+5V', gilbert_cell_envelope.gnd, 'DC 5')
gilbert_cell_envelope.V('6', '+2V', gilbert_cell_envelope.gnd, 'DC -7.5')
gilbert_cell_envelope.V('7', '+4V', gilbert_cell_envelope.gnd, 'DC 0')

simulator = gilbert_cell_envelope.simulator(temperature=25, nominal_temperature=25)
analysis_gilbert_cell_envelope = simulator.transient(step_time=1@u_us, end_time=20@u_ms)

In [ ]:
#simulate ring modulation
gilbert_am = parser.build_circuit(ground=5)
gilbert_am.include(spice_library['BC547B'])
gilbert_am.include(spice_library['TL072'])

gilbert_am.V('1', '+15V', gilbert_am.gnd, 'DC 15')
gilbert_am.V('2', '-15V', gilbert_am.gnd, 'DC -15')
gilbert_am.V('3', 'AUDIO_IN', gilbert_am.gnd, 'DC 0V AC 0V SIN(0 10m 1k)')
gilbert_am.V('4', 'CV_IN', gilbert_am.gnd, 'DC 0V AC 0V SIN(20m 20m 100)')
gilbert_am.V('5', '+5V', gilbert_am.gnd, 'DC 5')
gilbert_am.V('6', '+2V', gilbert_am.gnd, 'DC -7.5')
gilbert_am.V('7', '+4V', gilbert_am.gnd, 'DC 0')

simulator = gilbert_am.simulator(temperature=25, nominal_temperature=25)
analysis_gilbert_cell_am = simulator.transient(step_time=1@u_us, end_time=20@u_ms)

In [ ]:
#simulate ring modulation
gilbert_ring = parser.build_circuit(ground=5)
gilbert_ring.include(spice_library['BC547B'])
gilbert_ring.include(spice_library['TL072'])

gilbert_ring.V('1', '+15V', gilbert_ring.gnd, 'DC 15')
gilbert_ring.V('2', '-15V', gilbert_ring.gnd, 'DC -15')
gilbert_ring.V('3', 'AUDIO_IN', gilbert_ring.gnd, 'DC 0V AC 0V SIN(0 10m 1k)')
gilbert_ring.V('4', 'CV_IN', gilbert_ring.gnd, 'DC 0V AC 0V SIN(0 20m 100)')
gilbert_ring.V('5', '+5V', gilbert_ring.gnd, 'DC 5')
gilbert_ring.V('6', '+2V', gilbert_ring.gnd, 'DC -7.5')
gilbert_ring.V('7', '+4V', gilbert_ring.gnd, 'DC 0')

simulator = gilbert_ring.simulator(temperature=25, nominal_temperature=25)
analysis_gilbert_ring= simulator.transient(step_time=1@u_us, end_time=20@u_ms)

In [ ]:
fig, (ax0, ax1, ax2) = plt.subplots(nrows=1, ncols=3, sharex=False, figsize=(18, 6))

#plot the image
#img_diffpair_path = directory_path.joinpath('gilbert_cell', 'g2074.png')
#img_gilbert = mpimg.imread(str(img_diffpair_path))
#im = ax0.imshow(img_gilbert)
#ax0.axis('off')
#ax0.set_title('gilbert cell.', y=-0.2)

#plot the ringmodulation
ax0.plot(analysis_gilbert_ring['CV_IN'].abscissa*1000, analysis_gilbert_ring['CV_IN']*250, c='grey')  # modulating
ax0.plot(analysis_gilbert_ring['OUT'].abscissa*1000, analysis_gilbert_ring['OUT'], c='orange')  # differential output

ax0.grid()
ax0.set_xlabel('t [ms]')
ax0.set_ylabel('[V]')
ax0.set_title('DSB-SC (ringmodulation).', y=-0.2)

#plot the envelope
ax1.plot(analysis_gilbert_cell_envelope['CV_IN'].abscissa*1000, analysis_gilbert_cell_envelope['CV_IN']*250, c='grey')  # modulating
ax1.plot(analysis_gilbert_cell_envelope['OUT'].abscissa*1000, analysis_gilbert_cell_envelope['OUT'], c='orange')  # differential output

ax1.grid()
ax1.set_xlabel('t [ms]')
ax1.set_ylabel('[V]')
ax1.set_title('output with an envelope control voltage.', y=-0.2)

#plot the ring modulation
ax2.plot(analysis_gilbert_cell_am['CV_IN'].abscissa*1000, analysis_gilbert_cell_am['CV_IN']*250, c='grey')  # modulating
ax2.plot(analysis_gilbert_cell_am['OUT'].abscissa*1000, analysis_gilbert_cell_am['OUT'], c='orange')  # differential 
ax2.legend(('Vinput b  [V]', 'Vinput a [V]', 'Vout [V]'), loc=(.8, .8))
ax2.grid()
ax2.set_xlabel('t [ms]')
ax2.set_ylabel('[V]')
ax2.set_title('output with a sine control voltage.', y=-0.2)

plt.show()


this is not real amplitude modulation. when the signal is negative the phase of the output is inverted. this can be adjusted by the bias voltage of the control voltage. the control voltage has to be positive at all time. 


## *construction*

for the final circuit input and output buffering and biasing is needed. the buffering is done with opamps. 


first we design the input stage. the input voltage for the X signal should be 10mV and 20mV for the control voltage.


In [ ]:
#load the kicad schema
directory_path = Path(os.path.abspath('')).resolve()
kicad_netlist_path = directory_path.joinpath('main', 'main.cir')
parser = SpiceParser(path=str(kicad_netlist_path))


In [ ]:
#ringmodulation
ringmod_schema = parser.build_circuit(ground=5)
ringmod_schema.include(spice_library['TL072c'])
ringmod_schema.include(spice_library['AD633'])

ringmod_schema.V('1', '+15V', ringmod_schema.gnd, 'DC 15')
ringmod_schema.V('2', '-15V', ringmod_schema.gnd, 'DC -15')
ringmod_schema.V('3', 'IN_Xa', ringmod_schema.gnd, 'DC 5V AC 5V SIN(0V 5V 1k)')
ringmod_schema.V('4', 'IN_Ya', ringmod_schema.gnd, 'DC 5V AC 5V SIN(0V 5V 100)')
ringmod_schema.V('5', 'IN_Za', ringmod_schema.gnd, 'DC 0V AC 0V')

simulator = ringmod_schema.simulator(temperature=25, nominal_temperature=25)
analysis_ringmod = simulator.transient(step_time=10@u_us, end_time=20@u_ms)

In [ ]:
#amplitude modulation
am_schema = parser.build_circuit(ground=5)
am_schema.include(spice_library['TL072c'])
am_schema.include(spice_library['AD633'])

am_schema.V('1', '+15V', am_schema.gnd, 'DC 15')
am_schema.V('2', '-15V', am_schema.gnd, 'DC -15')
am_schema.V('3', 'IN_Xa', am_schema.gnd, 'DC 10V AC 5V SIN(0V 5V 1k)')
am_schema.V('4', 'IN_Ya', am_schema.gnd, 'DC 5V AC 5V SIN(0V 5V 100)')
am_schema.V('5', 'IN_Za', am_schema.gnd, 'DC 5V AC 5V SIN(0V 2.5V 1k)')

simulator = am_schema.simulator(temperature=25, nominal_temperature=25)
analysis_am = simulator.transient(step_time=10@u_us, end_time=20@u_ms)

In [ ]:
#vca
vca_schema = parser.build_circuit(ground=5)
vca_schema.include(spice_library['TL072c'])
vca_schema.include(spice_library['AD633'])

vca_schema.V('1', '+15V', vca_schema.gnd, 'DC 15')
vca_schema.V('2', '-15V', vca_schema.gnd, 'DC -15')
vca_schema.V('3', 'IN_Xa', vca_schema.gnd, 'DC 10V AC 5V SIN(0V 5V 1k)')
vca_schema.V('4', 'IN_Ya', vca_schema.gnd, 'DC 5V AC 5V PULSE(0V 10V 1m 1m 15m 1m)')
vca_schema.V('5', 'IN_Za', vca_schema.gnd, 'DC 0V AC 0V')

simulator = vca_schema.simulator(temperature=25, nominal_temperature=25)
analysis_vca = simulator.transient(step_time=10@u_us, end_time=20@u_ms)

In [ ]:
fig, (ax0, ax1, ax2) = plt.subplots(nrows=1, ncols=3, sharex=False, figsize=(18, 6))

#plot the ringmodulation
ax0.plot(analysis_ringmod['IN_Ya'].abscissa*1000, analysis_ringmod['IN_Ya'], c='grey', linestyle=':')
ax0.plot(analysis_ringmod['OUT_a'].abscissa*1000, analysis_ringmod['OUT_a'], c='orange') 
ax0.grid()
ax0.set_xlabel('t [ms]')
ax0.set_ylabel('[V]')
ax0.set_title('DSB-SC (ringmodulation).', y=-0.2)

#plot the amplitude modulation
ax1.plot(analysis_am['IN_Ya'].abscissa*1000, analysis_am['IN_Ya'], c='grey', linestyle=':') 
ax1.plot(analysis_am['OUT_a'].abscissa*1000, analysis_am['OUT_a'], c='orange') 
ax1.grid()
ax1.set_xlabel('t [ms]')
ax1.set_ylabel('[V]')
ax1.set_title('amplitude modulation', y=-0.2)

#plot the envelope
ax2.plot(analysis_vca['IN_Ya'].abscissa*1000, analysis_vca['IN_Ya'], c='grey', linestyle=':')
ax2.plot(analysis_vca['OUT_a'].abscissa*1000, analysis_vca['OUT_a'], c='orange') 
ax2.grid()
ax2.set_xlabel('t [ms]')
ax2.set_ylabel('[V]')
n = ax2.set_title('envelope control voltage.', y=-0.2)


In [ ]:
import unittest

class TestInputVoltages(unittest.TestCase):
    
    def test_ringmod(self):
        self.assertAlmostEqual(4.95, np.max(np.array(analysis_ringmod.OUT_a)), places=1, msg='ringmod input max voltage x')
        self.assertAlmostEqual(0, np.average(np.array(analysis_ringmod.OUT_a)), places=1, msg='ringmod input average voltage x')

    def test_am(self):
        self.assertAlmostEqual(9.95, np.max(np.array(analysis_am.OUT_a)), places=1, msg='am input max voltage x')
        self.assertAlmostEqual(0, np.average(np.array(analysis_am.OUT_a)), places=1, msg='am input average voltage x')

    def test_vca(self):
        self.assertAlmostEqual(10, np.max(np.array(analysis_vca.OUT_a)), places=1, msg='vca input voltage x')
        self.assertAlmostEqual(0.01, np.average(np.array(analysis_vca.OUT_a)), places=1, msg='vca input voltage y')



In [ ]:
unittest.main(argv=[''], verbosity=2, exit=False)

## *references*

- [VCA Techniques Investigated][1] Rod Elliott (ESP)
- [Popular Electronics][2] Keyiing and VCA citcuits for electronic music instruments 
- [VCA-1][3] Thomas Henry CA3080 vca
- [VCA-3][4] René Schmitz differential pair vca
- [#223][5]: Basics of the Gilbert Cell | Analog Multiplier | Mixer | Modulator
- [#224][6]: AM & DSB-SC Modulation with the Gilbert Cell
- [Analog Devices][8]: Analog multiplier application guide 
- [AD633][9]: Datasheet 


[1]: https://sound-au.com/articles/vca-techniques.html
[2]: https://tinaja.com/glib/pop_elec/mus_keying_vca_1+2_75.pdf
[3]: https://www.birthofasynth.com/Thomas_Henry/Pages/VCA-1.html
[4]: https://www.schmitzbits.de/vca3.png
[5]: https://www.youtube.com/watch?v=7nmmb0pqTU0&t=2s
[6]: https://www.youtube.com/watch?v=38OQub2Vi2Q
[7]: http://www.ecircuitcenter.com/Circuits/BJT_Diffamp1/BJT_Diffamp1.htm
[8]: https://www.analog.com/media/en/training-seminars/design-handbooks/ADI_Multiplier_Applications_Guide.pdf
[9]: https://www.analog.com/media/en/technical-documentation/data-sheets/AD633.pdf



